In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install azure-cosmos

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install windows-tools

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Nathalia 

# Importar módulos y funciones necesarios
# os : interactua con el sistema operativo
# request : para hacer solicitudes HTTP
# json : para generar UUIDs únicos
# azure.cosmos : interactua con Azure Cosmos DB
# matplotlib : para visualización de datos
import os 
import requests  
import json 
import uuid 
from azure.cosmos import CosmosClient 
import matplotlib.pyplot as plt  
from windows_tools.installed_software import get_installed_software  

# Información de Azure Cosmos DB
settings = {
    'host': os.environ.get('ACCOUNT_HOST', 'https://cosmostuchis.documents.azure.com:443/'),
    'master_key': os.environ.get('ACCOUNT_KEY', 'xziB71AvZnnDxvaquvjWP386zNpUSyXMa21jrJUTxRTjydcyPgZc0vb7Qa2HyPOjIxtUPOsnprYCACDbQh67fA=='),
    'database_id': os.environ.get('COSMOS_DATA', 'cosmostuchis'),
    'container_id': os.environ.get('COSMOS_CONTAINER', 'CVE')
}

# Variables para la configuración de Cosmos DB
HOST = settings['host']  
MASTER_KEY = settings['master_key']  
DATABASE_ID = settings['database_id']  
CONTAINER_ID = settings['container_id']  

# Función para la interacción con una base de datos Cosmos DB para guardar información sobre vulnerabilidades de software
def save_to_cosmos_db(software, vulnerabilities):
    
  
    client = CosmosClient(HOST, credential=MASTER_KEY)
   
    database = client.get_database_client(DATABASE_ID)
    
    container = database.get_container_client(CONTAINER_ID)
    
    vulnerabilities_info = {
        'id': str(uuid.uuid4()),  
        'Software': software,
        'Vulnerabilities': vulnerabilities
    }
   
    container.create_item(body=vulnerabilities_info)
    # Imprimir 
    print(json.dumps({
        'message': 'Se ha guardado exitosamente en Cosmos DB'
    }, ensure_ascii=False))
    


# Función que busca las vulnerabilidades del software utilizando la Api Vulners.
def buscar_vulnerabilidades(software):
    
    url = "https://vulners.com/api/v3/search/lucene/"
    headers = {"Content-Type": "application/json"}
    data = {
        "query": software,
        "size": 50
    }
    #  Realiza una solicitud HTTP POST a una URL 
    response = requests.post(url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        return response.json()  
    else:
        return None  
# Función para obtener detalles de CVE
def obtener_detalles_cve(cve):
    
    url = f'https://cve.circl.lu/api/cve/{cve}'
   # Realizar una solicitud GET para los detalles de CVE.
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()  
    else:
        return None  
# Función para clasificar CVSS
def clasificar_cvss(cvss):
    #Clasifica el puntaje de CVSS
    if cvss is None:
        return "None", 0.0
    elif cvss < 4.0:
        return "Low", cvss
    elif cvss < 7.0:
        return "Medium", cvss
    elif cvss < 9.0:
        return "High", cvss
    else:
        return "Critical", cvss
# Contador de softwares con vulnerabilidades
vulnerable_software_count = 0
#Realiza un escaneo de software en busca de vulnerabilidades
for software in get_installed_software():
    if vulnerable_software_count >= 10:  # Detiene el escaneo en 10
        break
    name = software['name']
    if name:  
        first_two_words = ' '.join(name.split()[:2])
        print(f"Investigando vulnerabilidades de: {first_two_words}")
        
        vulnerabilidades = buscar_vulnerabilidades(first_two_words)
        if vulnerabilidades:
            software_vulnerabilities = []
            
            for vulnerabilidad in vulnerabilidades['data']['search']:
                cve = vulnerabilidad['_source']['id']
                detalles_cve = obtener_detalles_cve(cve)
                if detalles_cve:
                    cvss_classification, cvss_score = clasificar_cvss(detalles_cve['cvss'])
                   
                    info_vulnerabilidad = {
                        "CVE": cve,
                        "Summary": detalles_cve['summary'],
                        "Published": detalles_cve['Published'],
                        "Last modification": detalles_cve['last-modified'],
                        "CVSS": cvss_score,
                        "CVSS Classification": cvss_classification
                    }
                    software_vulnerabilities.append(info_vulnerabilidad)
            if software_vulnerabilities:
                save_to_cosmos_db(first_two_words, software_vulnerabilities)  
                vulnerable_software_count += 1
                
        else:
            print(f"No se obtuvo más información para las vulnerabilidades de {first_two_words}.")



# Función para obtener los datos de la base de datos
def get_vulnerability_data():
    
    client = CosmosClient(HOST, credential=MASTER_KEY)
    database = client.get_database_client(DATABASE_ID)
    container = database.get_container_client(CONTAINER_ID)
    query = "SELECT * FROM c"
    items = list(container.query_items(query=query, enable_cross_partition_query=True))
    return items


# Función para obtener los datos de las vulnerabilidades de la base de datos
# luego calcular la distribución de CVSS de esas vulnerabilidades, es decir, 
# contar cuántas vulnerabilidades hay en cada nivel de clasificación de CVSS.
def get_vulnerability_data():
    
    client = CosmosClient(HOST, credential=MASTER_KEY)
    database = client.get_database_client(DATABASE_ID)
    container = database.get_container_client(CONTAINER_ID)
    query = "SELECT * FROM c"
    items = list(container.query_items(query=query, enable_cross_partition_query=True))
    return items


vulnerability_data = get_vulnerability_data()

# Distribución de CVSS
cvss_counts = {"None": 0, "Low": 0, "Medium": 0, "High": 0, "Critical": 0}

for data in vulnerability_data:
    vulnerabilities = data['Vulnerabilities']
    for vulnerability in vulnerabilities:
        cvss_classification = vulnerability["CVSS Classification"]
        cvss_counts[cvss_classification] += 1

# Se encuentra los datos de gráfico de barras y gráfico de torta.

# Gráfico de barras
categorias = ['Critical', 'High', 'Medium', 'Low', 'None']
valores = [cvss_counts[cat] for cat in categorias]

# Colores 
colores = ['black', 'red', 'orange', 'green', 'blue']

# Creación
plt.figure(figsize=(10, 5))
plt.bar(categorias, valores, color=colores)

# Título y etiquetas 
plt.title('Impacto por criticidad de vulnerabilidades')
plt.xlabel('Criticidad')
plt.ylabel('Impacto')

# Mostrar 
plt.show()

# Gráfico de tarta
etiquetas = categorias
tamaños = valores

# Creación
plt.figure(figsize=(8, 8))
plt.pie(tamaños, labels=etiquetas, autopct='%1.1f%%', colors=colores, textprops={'color': '#030E00'})
plt.title('Distribución de vulnerabilidades por criticidad')

# Verificación para que se muestre como un círculo
plt.axis('equal')

# Mostrar 
plt.show()


Investigando vulnerabilidades de: Arduino IDE
{"message": "Se ha guardado exitosamente en Cosmos DB"}
Investigando vulnerabilidades de: Git
{"message": "Se ha guardado exitosamente en Cosmos DB"}
Investigando vulnerabilidades de: GDR 1105
Investigando vulnerabilidades de: GDR 1110
Investigando vulnerabilidades de: GDR 1115
Investigando vulnerabilidades de: MATLAB R2023b
Investigando vulnerabilidades de: Microsoft SQL
{"message": "Se ha guardado exitosamente en Cosmos DB"}
Investigando vulnerabilidades de: Microsoft SQL
{"message": "Se ha guardado exitosamente en Cosmos DB"}
Investigando vulnerabilidades de: GlassFish Server
{"message": "Se ha guardado exitosamente en Cosmos DB"}
Investigando vulnerabilidades de: NetBeans IDE
{"message": "Se ha guardado exitosamente en Cosmos DB"}
Investigando vulnerabilidades de: Notepad++ (64-bit
{"message": "Se ha guardado exitosamente en Cosmos DB"}
Investigando vulnerabilidades de: Aplicaciones de
Investigando vulnerabilidades de: Microsoft OneDriv